In [1]:
import pandas as pd
import numpy as np
import re

datasets_dir = 'C:/Courses/1 - CS506 - Tools for Data Science/Project/Data/CSV/'

In [2]:
contributions_2016 = 'contributions-2015-2016.csv'
df_contributions_2016 = pd.read_csv(datasets_dir + contributions_2016, encoding = 'latin1')

In [3]:
statereps_2016 = 'State_Reps_2015-2016.csv'
df_reps_2016 = pd.read_csv(datasets_dir + statereps_2016, encoding = 'latin1')

In [4]:
contributions_2018 = 'contributions-2017-2018.csv'
df_contributions_2018 = pd.read_csv(datasets_dir + contributions_2018, encoding = 'latin1')

C:\Users\WINPC\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
statereps_2018 = 'State_Reps_2017-2018.csv'
df_reps_2018 = pd.read_csv(datasets_dir + statereps_2018, encoding = 'latin1')

In [6]:
import pandas as pd

# Get list of all elected Candidates from 2015-2018
reps = [df_reps_2016, df_reps_2018]
df_reps = pd.concat(reps, sort=True)
state_reps = list(df_reps['Representative'].unique())

# Deallocate memory
reps = []
df_reps_2016 = []
df_reps_2018 = []

In [7]:
import pandas as pd

# Combine all contributions from 2015-2018
contributions = [df_contributions_2016, df_contributions_2018]
df_contributions = pd.concat(contributions, sort=True)

# Deallocate memory
contributions = []
df_contributions_2016 = []
df_contributions_2018 = []

In [8]:
display(len(state_reps))
display(df_contributions.shape)

193

(894666, 17)

In [9]:
df_2018_real = df_contributions.loc[df_contributions.Full_Name.isin(state_reps)]

In [10]:
# Creating new Donor_Name column that will allow easier analysis here
# and moving forward

import math

def donor_name(first_name, last_name):
    donor_name = ''
    if (type(first_name) == str):
        donor_name += first_name.upper().replace(":-,.'", ' ') + ' '
    if (type(last_name) == str):
        donor_name += last_name.upper().replace(":-,.'", ' ')
    return donor_name

df_2018_real['Donor_Name'] = df_2018_real[['First_Name', 'Last_Name']].apply(lambda x: donor_name(x[0], x[1]), axis=1)

C:\Users\WINPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
import re
import numpy as np

# Regular Expression Search for each Industry

# Law Enforcement
law_enforcement_keywords = ['law enforcement', 'district attorney', 'prosecutor', 'detective', 'trooper', 'probation officer', 'sheriff', 'correction', 'prison', 'patrolm[ae]n']
law_enforcement_regex = ''

for line in law_enforcement_keywords:
    law_enforcement_regex += '(' + line +')|'

law_enforcement_regex += '(police)'
# display(regex_expr)

In [12]:
# Education
education_keywords = ['teacher', 'educator', 'university', 'school', 'college', 'tutor', 'tuition', 'academ(y|ic)']
education_regex = ''

for line in education_keywords:
    education_regex += '(' + line +')|'

education_regex += '(education)'
# display(regex_expr)

In [13]:
# Real Estate
realestate_keywords = ['realtor', 'realty', 'housing', 'leasing', '[^(it)|^(software)|^(data)] developer', '[^(it)|^(software)|^(data)] architect', 'construction', 'lodging', 'propert(y|ies)', 'residential', 'contractor']
realestate_regex = ''

for line in realestate_keywords:
    realestate_regex += '(' + line +')|'

realestate_regex += '(real estate)'
# display(regex_expr)

In [14]:
# Healthcare
# Note - 'opthalm', 'urolog', 'chiropract', 'psychiatr' are added to match root words of medical specializations
healthcare_keywords = ['health care', 'health', 'medical', 'hospital', 'nurse', 'rehab', 'nursing', 'clinic', 'doctor', 'psychologist', 'surgeon', 'dental', 'dentist', 'physician', 'opthalm', 'urolog', 'pharmacist', 'eye care', 'eye center', 'chiropract', 'psychiatr']
healthcare_regex = ''

for line in healthcare_keywords:
    healthcare_regex += '(' + line +')|'

healthcare_regex += '(healthcare)'
# display(regex_expr)

In [15]:
# BioTech / Pharma
biotech_keywords = ['genetech', 'bioscience']
biotech_regex = ''

for line in biotech_keywords:
    biotech_regex += '(' + line +')|'

biotech_regex += '(bio science)'
# display(regex_expr)

In [16]:
def find_industry(donor_name, occupation, employer): 
    text = str(donor_name) + ' ' + str(occupation) + ' ' + str(employer)
    text = text.lower().replace(":-,.'", '')
#     display(text)
    if(re.search(law_enforcement_regex, text)):
        return 'Law Enforcement'
    if (re.search(education_regex, text)):
        return 'Education'
    if (re.search(realestate_regex, text)):
        return 'Real Estate'
    if (re.search(healthcare_regex, text)):
        return 'Healthcare'
    if (re.search(biotech_regex, text)):
        return 'Biotech/Pharma'
    # Case no match found
    return np.nan

In [17]:
# PAC keywords
PAC_keywords = ['\Wpac\W', 'pol action', 'political action', 'committee', 'action']
PAC_regex = ''

for line in PAC_keywords:
    PAC_regex += '(' + line +')|'

PAC_regex += '(real estate)'
# display(regex_expr)

In [18]:
# Union Keywords
union_keywords = ['\Wunion\W', 'association']
union_regex = ''

for line in union_keywords:
    union_regex += '(' + line +')|'

union_regex += '(union pac)'
# display(regex_expr)

In [19]:
from fuzzywuzzy import fuzz

def fuzzy_compare(str1, str2):
#     display(str1 + ' == ' +str2)#debug
    ratio = fuzz.ratio(str1.lower().replace(":-,.'", ''), str2.lower().replace(":-,.'", ''))
#     display(ratio)#debug
    if (ratio > 80):
        return True
    else:
        return False

In [26]:
import pandas as pd

df_state_reps = pd.DataFrame(state_reps, columns = ['Representative'])
# display(df_state_reps['Representative'].head())

def find_PAC(donor_name, candidate_name): 
    text = ' ' + str(donor_name) + ' '
    text = text.lower().replace(":-,.'", ' ')
#     display(text)
    # Note - Union checking must come before PAC because of "union pac"
    if(re.search(union_regex, text)):
        return 'Union'
    if(re.search(PAC_regex, text)):
        return 'PAC'
    
#     candidateFound = df_state_reps['Representative'].apply(lambda x: fuzzy_compare(donor_name, x))
    candidateFound = fuzzy_compare(donor_name, candidate_name)
#     display(candidateFound)
    if (candidateFound):
        return 'Candidate'
    # Case no match found
    return np.nan

In [21]:
df_2018_real['Industry'] = df_2018_real[['Donor_Name', 'Occupation', 'Employer']].apply(lambda x: find_industry(x[0], x[1], x[2]), axis=1)

# df_2018_real['Industry'] = df_2018_real[df_2018_real.Occupation.notnull()]['Occupation'].apply(find_industry)
# df_2018_real['Industry'] = df_2018_real[df_2018_real.Employer.notnull()]['Employer'].apply(find_industry)

C:\Users\WINPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
df_2018_real['PAC'] = df_2018_real[['Donor_Name', 'Full_Name']].apply(lambda x: find_PAC(x[0], x[1]), axis=1)

C:\Users\WINPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
# df_2018_real.loc[df_2018_real.PAC.notnull()]

In [29]:
df_2018_real.to_csv('C:/Courses/1 - CS506 - Tools for Data Science/Project/Data/Analysis Report/contribution_processed.csv')

In [25]:
# df_2018_real.loc[df_2018_real.Full_Name=='Robert A. DeLeo']